In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
import numpy
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import glob
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
# fix random seed for reproducibility
numpy.random.seed(7)

Using TensorFlow backend.


In [2]:
#load dataset
Friday = pd.read_csv("Friday-Afternoon-DDos.csv")
Friday_ps = pd.read_csv("Friday-Afternoon-PortScan.csv")
Friday_bot = pd.read_csv("Friday-Morning-bot.csv")
Monday = pd.read_csv("Monday-Normal.csv")
Thursday_i = pd.read_csv("Thursday-Afternoon-Infilteration.csv")
# Thursday_w = pd.read_csv("Thursday-Morning-WebAttacks.csv")
Tue = pd.read_csv("Tuesday-WorkingHours-FTP-SSH.csv")
#Wed = pd.read_csv("Wednesday-DoS.csv")

df = pd.DataFrame()
df = df.append(Friday)
df = df.append(Friday_ps)
df = df.append(Friday_bot)
df = df.append(Monday)
df = df.append(Thursday_i)
#df.append(Thursday_w)
df = df.append(Tue)
# df = df.append(Wed)
df.head()


#genfromtxt handles the datatype as we are using both string and float
# scaler = MinMaxScaler()
# scaled_data = scaler.fit_transform(data)



C:\Users\Susmit\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Susmit\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Source IP,Source Port,Destination IP,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.10.16,41936.0,199.244.48.55,443.0,6.0,143347.0,47.0,60.0,1325.0,108751.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,192.168.10.16,42970.0,54.210.195.63,80.0,6.0,50905.0,1.0,1.0,0.0,0.0,...,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,192.168.10.16,41944.0,199.244.48.55,443.0,6.0,143899.0,46.0,58.0,1325.0,110185.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,192.168.10.17,12886.0,192.168.10.3,53.0,17.0,313.0,2.0,2.0,90.0,206.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,192.168.10.16,41942.0,199.244.48.55,443.0,6.0,142605.0,45.0,58.0,1325.0,108751.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


In [3]:
data=df.values
y = data[:,82]
print(y)
df[['Source IP1','Source IP2','Source IP3', 'Source IP4']] = df[' Source IP'].str.split('.', expand=True)
df[['Destination IP1','Destination IP2','Destination IP3', 'Destination IP4']] = df[' Destination IP'].str.split('.', expand=True)
# print(test)

#df[['Source IP1','Source IP2','Source IP3', 'Source IP4', 'Destination IP1','Destination IP2','Destination IP3', 'Destination IP4', 'Flow Bytes/s', ' Flow Packets/s']] = df[['Source IP1','Source IP2','Source IP3', 'Source IP4', 'Destination IP1','Destination IP2','Destination IP3', 'Destination IP4', 'Flow Bytes/s', ' Flow Packets/s']].apply(pd.to_numeric, errors='ignore')

df = df.drop(' Source IP', axis=1)
df = df.drop(' Destination IP', axis=1)
df = df.drop(' Label', axis=1)

#df_new = df.values
# print(test)
#x = df_new[:,0:88]
# print(x)

['BENIGN' 'BENIGN' 'BENIGN' ..., 'BENIGN' 'BENIGN' 'BENIGN']


In [4]:
df[['Source IP1','Source IP2','Source IP3', 'Source IP4', 'Destination IP1','Destination IP2','Destination IP3', 'Destination IP4']] = df[['Source IP1','Source IP2','Source IP3', 'Source IP4', 'Destination IP1','Destination IP2','Destination IP3', 'Destination IP4']].apply(pd.to_numeric)

In [ ]:
df.i

In [10]:
df.iloc[618]

 Source Port                         443
 Destination Port                  55043
 Protocol                              6
 Flow Duration                         0
 Total Fwd Packets                     2
 Total Backward Packets                0
Total Length of Fwd Packets           12
 Total Length of Bwd Packets           0
 Fwd Packet Length Max                 6
 Fwd Packet Length Min                 6
 Fwd Packet Length Mean                6
 Fwd Packet Length Std                 0
Bwd Packet Length Max                  0
 Bwd Packet Length Min                 0
 Bwd Packet Length Mean                0
 Bwd Packet Length Std                 0
Flow Bytes/s                    Infinity
 Flow Packets/s                 Infinity
 Flow IAT Mean                         0
 Flow IAT Std                          0
 Flow IAT Max                          0
 Flow IAT Min                          0
Fwd IAT Total                          0
 Fwd IAT Mean                          0
 Fwd IAT Std    

In [7]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1967674 entries, 0 to 445908
Data columns (total 88 columns):
 Source Port                    float64
 Destination Port               float64
 Protocol                       float64
 Flow Duration                  float64
 Total Fwd Packets              float64
 Total Backward Packets         float64
Total Length of Fwd Packets     float64
 Total Length of Bwd Packets    float64
 Fwd Packet Length Max          float64
 Fwd Packet Length Min          float64
 Fwd Packet Length Mean         float64
 Fwd Packet Length Std          float64
Bwd Packet Length Max           float64
 Bwd Packet Length Min          float64
 Bwd Packet Length Mean         float64
 Bwd Packet Length Std          float64
Flow Bytes/s                    object
 Flow Packets/s                 object
 Flow IAT Mean                  float64
 Flow IAT Std                   float64
 Flow IAT Max                   float64
 Flow IAT Min                   float64
Fwd IAT To

In [7]:
# values = array(y)
# print(values)

# # integer encode
# label_encoder = LabelEncoder()
# integer_encoded = label_encoder.fit_transform(values)
# print(integer_encoded)

# binary encode
# onehot_encoder = OneHotEncoder(sparse=False)
# integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
# onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
# print(onehot_encoded)

['BENIGN' 'BENIGN' 'BENIGN' ..., 'BENIGN' 'BENIGN' 'BENIGN']
[0 0 0 ..., 0 0 0]
[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]]


In [14]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_y= encoder.transform(y)

#convert integers to dummy variables (one hot encoding)
dummy_y = np_utils.to_categorical(encoded_y)

test_data = [encoded_y, dummy_y]

# df_test = pd.DataFrame(test_data)
# df_test.to_csv('test_data.txt', index = False, header = False)

print(encoded_y)

[0 0 0 ..., 0 0 0]


In [15]:
model = Sequential()
model.add(Dense(120, input_dim = 88,activation='relu'))
model.add(Dense(82, activation='relu'))
model.add(Dense(31, activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [16]:
#compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
model.fit(x,encoded_y, epochs=20, batch_size=5000)

Epoch 1/20
  50000/1967674 [..............................] - ETA: 3:32 - loss: nan - acc: 0.0000e+00

KeyboardInterrupt: 

In [ ]:


# evaluate the model
scores = model.evaluate(x, y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# evaluate the model
scores = model.evaluate(x, y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))